### 3. Classifiers

This notebook to classify the probability of fake news traveling from A to B.

In [1]:
ls data

ClassificationModelInput.csv* pol_agg_new.csv*
emergent.csv*                 politifact.csv*
fake_localcentralities.csv*   politifact_clean.csv*
key_mutuality_roshan.csv*     real_localcentralities.csv*
keys.csv*                     snopes.csv*


In [2]:
%pylab inline
import pandas as pd, pyprind

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score,make_scorer

Populating the interactive namespace from numpy and matplotlib


In [56]:
d=pd.read_csv('data/ClassificationModelInput.csv')
sum(sum(d.isnull())) #No NA
print(d.columns)

Index([u'Source', u'Destination', u'Monday', u'Elections', u'Military',
       u'Thursday', u'Religion', u'Friday', u'Saturday', u'TRUE', u'History',
       u'Tuesday', u'Wednesday', u'Sunday', u'page_url', u'Health Care',
       u'FALSE', u'jaccard_coeff_connection', u'Neighbor_connection',
       u'jaccard_coeff_common_destination', u'Neighbor_common_destination',
       u'jaccard_coeff_common_source', u'Neighbor_common_source',
       u'SourceSite_outdeg_real', u'DestSite_outdeg_real',
       u'SourceSite_outdeg_fake', u'DestSite_outdeg_fake',
       u'SourceSite_indeg_fake', u'DestSite_indeg_fake',
       u'DestSite_indeg_real', u'SourceSite_indeg_real', u'mutuality_ind',
       u'Label'],
      dtype='object')


In [57]:

d = d[d['Source'] == 'www.facebook.com']
#d = d[d['Destination'] == 'www.politifact.com']
d.shape

(167, 33)

In [58]:
features=['Monday', 'Elections', 'Military', 'Thursday',
       'Religion', 'Friday', 'Saturday','History', 'Tuesday',
       'Wednesday', 'Sunday', 'page_url', 'Health Care',
       'jaccard_coeff_connection', 'Neighbor_connection',
       'jaccard_coeff_common_destination', 'Neighbor_common_destination',
       'jaccard_coeff_common_source', 'Neighbor_common_source',
       'SourceSite_outdeg_fake', 'DestSite_outdeg_fake',
       'SourceSite_indeg_fake', 'DestSite_indeg_fake', 'mutuality_ind']

In [59]:
for col in ['Monday', 'Tuesday', 'Wednesday', 
                            'Thursday','Friday', 'Saturday', 'Sunday']:
    mean = d[col].mean()
    std = d[col].std()
    if std != 0:
        d[col] = d[col].apply(lambda row: (row - mean)/std).apply(lambda row: math.log(row+1))
    else:
        d[col] = 0
d['Label']=d['FALSE'].apply(lambda x: int(x>0))
#d['Label'] = d['FAKE']/(d['TRUE'] + d['FAKE'])
#Check for class imblanace
#print(float(sum(d['Label']==0))/len(d))

#Ready for model
X=d[features]
Y=d['Label']
d['Monday'].head()

0     2.175901
3    -0.513445
6     1.548472
9     0.676533
12    0.676533
Name: Monday, dtype: float64

In [60]:
#Random Forest, GBM

#Make AUC the performance metric
models = {'RF':{'model':RandomForestClassifier(),
                     'Params':{'n_estimators':range(50,70,10),
                              'max_depth':range(2,5), 'class_weight':['balanced']}},
                
         'GBM':{'model':GradientBoostingClassifier(),
                     'Params':{'n_estimators':range(50,100,10),
                               'max_depth':range(2,5),
                               'learning_rate':linspace(0.1,0.5,num=20)}}}

In [61]:
print('Number of features:', len(X.columns))
print('Number of records', len(d))

('Number of features:', 24)
('Number of records', 167)


In [62]:
def modeleva(cvfolds):
    allmodels={}
    for model in models.keys():
        grid = GridSearchCV(estimator=models[model]['model'], 
                            param_grid=models[model]['Params'],
                            cv=cvfolds, n_jobs=-1,scoring='roc_auc')
        modelstats={}
        modelstats['model']=grid.fit(X, Y)
        modelstats['best_estimator']=modelstats['model'].best_estimator_
        modelstats['best_score']=modelstats['model'].best_score_
        allmodels[model]=modelstats
        del modelstats
        print(model)
    results=pd.DataFrame.from_dict(allmodels).T.sort_values(by='best_score')
    results=results.reset_index()
    return(results)

In [63]:
allresults=modeleva(3)
allresults

RF
GBM


,index,best_estimator,best_score,model
0,RF,"(DecisionTreeClassifier(class_weight=None, cri...",0.869171,"GridSearchCV(cv=3, error_score='raise',\n ..."
1,GBM,([DecisionTreeRegressor(criterion='friedman_ms...,0.915578,"GridSearchCV(cv=3, error_score='raise',\n ..."


In [11]:
allresults['best_estimator'][0]

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=2, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [12]:
bestmodel

NameError: name 'bestmodel' is not defined

In [77]:
k='you are a fucking asshole'.decode('unicode')

LookupError: unknown encoding: unicode

In [80]:
from nltk.sentiment.util import demo_subjectivity
from nltk.sentiment import SentimentAnalyzer 
demo_sent_subjectivity('you are a fucking asshole')

Cannot find the sentiment analyzer you want to load.
Training a new one using NaiveBayesClassifier.
Training classifier


UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-1: ordinal not in range(128)

In [ ]:
from nltk.sentiment.util import demo_SENT_subjectivity

